<a href="https://colab.research.google.com/github/adarsh1927/Learning_Deep/blob/main/learning_deep.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# structure
- every lession has its own installations, imports and codes
- structure look like that
```
installations
imports
codes
```


# Lessons

## Creating a Model to detecting the birds

In [7]:
!pip install -Uqq duckduckgo_search  fastai

In [15]:
from duckduckgo_search import DDGS
from fastcore.all import *

from fastdownload import download_url
from fastai.vision.all import *

In [16]:
# search image from keyword and return a image url
# L is a fastcore.L object which provides a convenient way to manipulate list-like objects

# fuction return list of urls of images over internet
def search_images(keywords, max_images=200): return L(DDGS().images(keywords, max_results=max_images)).itemgot('image')

In [21]:
# lets check what kind of result we get.

# List of url
urls = search_images('bird photos', max_images=1)
# watch frist item
urls[0]

'https://images.pexels.com/photos/326900/pexels-photo-326900.jpeg?cs=srgb&dl=wood-flight-bird-326900.jpg&fm=jpg'